In [5]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import PyPDF2

# Load environment variables from .env file
load_dotenv()

# Get the API key from the environment variable
api_key = os.getenv("API_KEY")

client = OpenAI(api_key=api_key)

def summarize_pdf(pdf_path):
    # Open the PDF file
    with open(pdf_path, 'rb') as file:
        # Create a PDF reader object
        pdf_reader = PyPDF2.PdfReader(file)

        # Initialize an empty string to store the extracted text
        text = ""

        # Iterate over each page of the PDF
        for page in pdf_reader.pages:
            # Extract the text from the page
            text += page.extract_text()

    # Create a chat completion using the extracted text
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content":
             f"{text}"},
            {"role": "user", "content": "Read the paper, and summarize in 50 words each about the following:\n\n1. Background\n2. Challenge\n3. Current status\n4. Method\n5. Result\n6. Conclusion\n7. Outlook\n"}
        ]
    )

    # Open a file in write mode
    with open(f"{pdf_path}_output.txt", "w") as file:
        # Write the output to the file
        file.write(completion.choices[0].message.content)

    print(f"Written to {pdf_path}_output.txt")

# Example usage
pdf_path = "LIFT_paper.pdf"
summarize_pdf(pdf_path)

Written to LIFT_paper.pdf_output.txt
